In [1]:
from tensorflow.keras import preprocessing
import tensorflow as tf
import numpy as np
import boto3
import json
sagemaker = boto3.client('runtime.sagemaker')

with open('macron.txt') as f:
    text = f.read()

tokenizer = preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)
max_id = len(tokenizer.word_index)

def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [3]:
def next_char(body, temperature=1):
    X_new = preprocess([body])
    result = sagemaker.invoke_endpoint(
        EndpointName='tensorflow-inference-2021-05-26-13-22-36-181',
        ContentType='application/json',
        Body = json.dumps(X_new.numpy().tolist())
    )
    y_proba = result['Body'].read().decode()
    y_proba = json.loads(y_proba)['predictions']
    y_proba = np.array(y_proba)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(body, n_chars=50, temperature=1):
    for _ in range(n_chars):
        body += next_char(body, temperature)
    return body

print('t = .2', complete_text('Mes chers compatri', 150, .2))
print('t = .5', complete_text('Mes chers compatri', 150, .5))
print('t = 1', complete_text('Mes chers compatri', 150, 1))
print('t = 2', complete_text('Mes chers compatri', 150, 2))

t = .2 Mes chers compatriotes, c'est que c'est une règle de réduire les règles de ce qu'on a besoin de votre région de ce que je pense que c'est dur. je puis pas d'ailleurs d
t = .5 Mes chers comparations, parce que c'est une des régions pour que c'est privé pour se pour soi même d'ailleurs de région aux gens nos mois qui sont aussi, c'est une
t = 1 Mes chers comportements donc ces règles, toutes là vous avez prendre, c'est que je puis être pressaire, trois jours qu'on a bien aussi pour y avoir des donner du m
t = 2 Mes chers compoenerient freinedoyoe'mes, éconongerdier,. non? y, un jeune, en domjey vo, vous psocée. voilà,., et ùid'. à règlements ni d'aptal. notre fermer. les 
